In [10]:
import pandas as pd
import numpy as np
import sqlite3
import re

#read the CSV
dfRNAL = pd.read_csv('RNAL.csv', encoding='ISO-8859-1', sep=';')
dfAirbnb = pd.read_csv('insideairbnb/porto_06-2019_listings.csv')

dfAirbnb.license.replace('', np.nan, inplace=True)

dfAirbnb['license'].value_counts()
dfAirbnb['license'].isna().sum()

1379

In [8]:
def sonumeros(a):
    return re.sub('[^0-9]','', a)

def semano(a):
    return re.sub('/2019','',str(a))

#Uniformiza num de registo, remove duplicados, ...
print("Total: ", dfAirbnb.shape)
dfAirbnb['first_review']=pd.to_datetime(dfAirbnb['first_review'], format='%Y-%m-%d')
dfAirbnb['last_review']=pd.to_datetime(dfAirbnb['last_review'], format='%Y-%m-%d')
dfAirbnb = dfAirbnb[(dfAirbnb['last_review'] > '2019-01-01')]
dfAirbnb.dropna(subset=['first_review'], inplace=True)
print("Recentes: ", dfAirbnb.shape)
dfAirbnb.drop_duplicates(subset=['license'], inplace=True)
print("Sem duplicados: ", dfAirbnb.shape)
dfAirbnb.license = dfAirbnb.license.apply(semano)
dfAirbnb.license = dfAirbnb.license.apply(sonumeros)
dfAirbnb.license.replace('', np.nan, inplace=True)
dfAirbnb.dropna(subset=['license'], inplace=True)
print("Sem vazios: ", dfAirbnb.shape)



dfRNAL.Nregisto = dfRNAL.Nregisto.apply(sonumeros)



Total:  (10966, 106)
Recentes:  (7466, 106)
Sem duplicados:  (5413, 106)
Sem vazios:  (5407, 106)


In [22]:
dfAirbnb = dfAirbnb[dfAirbnb.neighbourhood_group_cleansed == 'PORTO']

#Join
dfJoin = dfAirbnb.set_index('license').join(dfRNAL.set_index('Nregisto'))

dfJoin[['first_review', 'Data Abertura Publico']]

,first_review,Data Abertura Publico
license,,
509067786,2011-11-14,NaN
6995,2011-04-26,2013-03-27
634,2012-05-14,2014-12-09
85541,2011-05-10,2018-10-15
11205,2011-07-25,2015-04-10
5380,2011-07-05,2014-03-28
17076,2014-08-21,2014-06-16
34971,2011-10-29,2016-09-20
6184,2011-06-29,2018-03-29


In [23]:
dfJoin.LATITUDE.replace('', np.nan, inplace=True)
dfJoin.LONGITUDE.replace('', np.nan, inplace=True)
dfJoin.loc[dfJoin.LATITUDE.isnull(), 'joinedLatitude'] = dfJoin.latitude
dfJoin.loc[dfJoin.LATITUDE.notnull(), 'joinedLatitude'] = dfJoin.LATITUDE
dfJoin.loc[dfJoin.LONGITUDE.isnull(), 'joinedLongitude'] = dfJoin.longitude
dfJoin.loc[dfJoin.LONGITUDE.notnull(), 'joinedLongitude'] = dfJoin.LONGITUDE

In [24]:
dfJoin.LATITUDE.notnull().sum()

3812

In [27]:
#CSV todo

dfJoin.to_csv('joined_output.csv')

In [28]:
ALsemregisto = dfJoin[dfJoin.LATITUDE.isna()]
print(ALsemregisto.shape)

(373, 133)


In [30]:
print(dfRNAL.shape)
print(dfAirbnb.shape)
print(dfJoin.shape)

(8317, 27)
(4185, 106)
(4185, 133)


In [11]:
dfRNAL.to_csv('RNAL_com_coords.csv')